In [2]:
import numpy as np
import pandas as pd
import featuretools as ft

In [58]:
# Uncomment this for local
pos = pd.read_csv('../data/POS_CASH_balance.csv')
test = pd.read_csv('../data/application_test.csv')
train_full = pd.read_csv('../data/application_train.csv')
bureau = pd.read_csv('../data/bureau.csv')
bureau_balance = pd.read_csv('../data/bureau_balance.csv')
cc_bal = pd.read_csv('../data/credit_card_balance.csv')
inst = pd.read_csv('../data/installments_payments.csv')
prev_app = pd.read_csv('../data/previous_application.csv')
sample_submission = pd.read_csv('../data/sample_submission.csv')
col_descr = pd.read_csv('../data/HomeCredit_columns_description.csv', encoding="ISO-8859-1")

In [59]:
train = train_full.drop('TARGET',axis = 1)
train_y = train_full['TARGET']

In [66]:
# Create new entityset
es = ft.EntitySet(id = 'train')
es = es.entity_from_dataframe(entity_id = 'train'
                              , dataframe = train
                              , index = 'SK_ID_CURR')
es = es.entity_from_dataframe(entity_id = 'bureau'
                             , dataframe = bureau
                             , index = 'SK_ID_BUREAU')
es = es.entity_from_dataframe(entity_id = 'bureau_bal'
                            , dataframe = bureau_balance
                              , make_index = True
                              , index = 'bureau_bal_id')
es = es.entity_from_dataframe(entity_id = 'pos'
                             , dataframe = pos
                             , make_index = True
                             , index = 'pos_id')
es = es.entity_from_dataframe(entity_id = 'cc_bal'
                             , dataframe = cc_bal
                             , make_index = True
                             , index = 'cc_bal_id')
es = es.entity_from_dataframe(entity_id = 'inst'
                             , dataframe = inst
                             , make_index = True
                             , index = 'inst_id')
es = es.entity_from_dataframe(entity_id = 'prev_app'
                             , dataframe = prev_app
                             , index = 'SK_ID_PREV')

# Create relationships
r_train_bureau = ft.Relationship(es['train']['SK_ID_CURR']
                                , es['bureau']['SK_ID_CURR'])
es = es.add_relationship(r_train_bureau)

r_bureau_bureau_bal = ft.Relationship(es['bureau']['SK_ID_BUREAU']
                                      , es['bureau_bal']['SK_ID_BUREAU'])
es = es.add_relationship(r_bureau_bureau_bal)

r_train_pos = ft.Relationship(es['train']['SK_ID_CURR']
                             , es['pos']['SK_ID_CURR'])
es = es.add_relationship(r_train_pos)

r_train_cc_bal = ft.Relationship(es['train']['SK_ID_CURR'],
                                es['cc_bal']['SK_ID_CURR'])
es = es.add_relationship(r_train_cc_bal)

r_train_pos = ft.Relationship(es['train']['SK_ID_CURR']
                             , es['pos']['SK_ID_CURR'])
es = es.add_relationship(r_train_pos)

r_train_inst = ft.Relationship(es['train']['SK_ID_CURR']
                             , es['inst']['SK_ID_CURR'])
es = es.add_relationship(r_train_inst)

r_train_prev_app = ft.Relationship(es['train']['SK_ID_CURR']
                             , es['prev_app']['SK_ID_CURR'])
es = es.add_relationship(r_train_prev_app)

es

2018-06-08 18:28:31,693 featuretools.entityset - WARNING    Not adding duplicate relationship: <Relationship: pos.SK_ID_CURR -> train.SK_ID_CURR>


Entityset: train
  Entities:
    train [Rows: 307511, Columns: 121]
    bureau [Rows: 1716428, Columns: 17]
    bureau_bal [Rows: 27299925, Columns: 4]
    pos [Rows: 10001358, Columns: 9]
    cc_bal [Rows: 3840312, Columns: 24]
    inst [Rows: 13605401, Columns: 9]
    prev_app [Rows: 1670214, Columns: 37]
  Relationships:
    bureau.SK_ID_CURR -> train.SK_ID_CURR
    bureau_bal.SK_ID_BUREAU -> bureau.SK_ID_BUREAU
    pos.SK_ID_CURR -> train.SK_ID_CURR
    cc_bal.SK_ID_CURR -> train.SK_ID_CURR
    inst.SK_ID_CURR -> train.SK_ID_CURR
    prev_app.SK_ID_CURR -> train.SK_ID_CURR

In [ ]:
# Create new features using specified primitives
features, feature_names = ft.dfs(entityset = es
                                , target_entity = 'train'
                                , agg_primitives = ['mean', 'max', 'last']
                                , trans_primitives = ['years', 'month', 'subtract', 'divide'])

In [ ]:
#feature tools taking about an hour so far